In [1]:
import torch
import re
import os
import json
import pickle
import openai
import pandas as pd
import numpy as np
import glob
from tqdm import tqdm
import ast
from dotenv import load_dotenv
import time

In [2]:
load_dotenv()
openai.api_key = os.environ['OPENAI_API']

In [3]:
def extract_present_traits(species, df):

    s = df.loc[species]
    GT_traits = list(s.where(s == 1).dropna().index)
    empty_traits = [F"{Entity}:" for (Entity, Value) in GT_traits]

    return GT_traits, empty_traits

### Trait Dicts

In [4]:
folder_traits = "../../../data/OpenAI/Traits/"

with open(F"{folder_traits}Andrei.json", 'r') as f:
  caribbean_traits_dict = json.load(f)

with open(F"{folder_traits}Andrei.json", 'r') as f:
  caribbean_traits_dict = json.load(f)

with open(F"{folder_traits}Andrei.json", 'r') as f:
  caribbean_traits_dict = json.load(f)

### DataFrames

In [5]:
root = "../../../data/OpenAI/DataFrames/"

file = "DF_Andrei.csv"
df_Andrei = pd.read_csv(F"{root}{file}", header=[0, 1], index_col=0)\
    .rename_axis('Species', axis='index')
df_Andrei_species = list(df_Andrei.index)


file = "DF_Daniel.csv"
df_Daniel = pd.read_csv(F"{root}{file}", header=[0, 1], index_col=0)\
    .rename_axis('Species', axis='index')\
    .dropna()
df_Daniel_species = list(df_Daniel.index)

file = "DF_Pierre.csv"
df_Pierre = pd.read_csv(F"{root}{file}", header=[0, 1], index_col=0)\
    .rename_axis('Species', axis='index')
df_Pierre_species = list(df_Pierre.index)

In [6]:
species_lst = df_Andrei.index
traits = list(caribbean_traits_dict.keys())

In [7]:
# Define the path to the directory where the prompts and results will be saved
folder_prompts = "../../../data/OpenAI/PromptsResults/ZeroShot/"

# Loop over species
for idx, species in (pbar := tqdm(enumerate(species_lst[0:5]), leave=False, position=0)):
    pbar.set_description(f"{idx}: {species}")
   
    # Replace spaces in the species name with underscores
    file_name = species.replace(' ', '_')

    # Create the question and options for the ChatGPT prompt
    question = F"Can you create semantic triples for the following traits {traits}, for the species '{species}' and return the answer as a list of of tuples for easy parsing?"
    user_content = F"{question}"

    # Check if file is already there (OpenAI Outage)
    if os.path.exists(F"{folder_prompts}{file_name}.json"):
        continue

    # # Create the messages to send to the ChatGPT API
    messages = [
        {"role": "user", "content": user_content}
        ]
    # Call the ChatGPT API to generate a completion for the prompt
    completion = openai.ChatCompletion.create(
        model = "gpt-3.5-turbo",
        messages = messages,
    )

    # Lower pressure on API?
    time.sleep(2)

    # Save the completion to a JSON file with the file name in the directory
    with open(F"{folder_prompts}{file_name}.json", 'w') as fp:
        json.dump(completion, fp)
            